In [1]:
# 1 imports
import pandas as pd
from matplotlib import pyplot as plt
import os

In [2]:
# 2 runfile directories
runfile_directory = 'Runfiles/'
results_directory = 'Results/'
template_file = 'template.txt'

In [16]:
# 3 parameter dict and template writing functions
parameter_dict = {'*fname': ['Python/Results/'],
                  'steps': ['100'],
                 'reps': ['1'],
                 'letter_grades' : ['true', 'false'],
                  'prefix' : ["new_model_test"],
                  'grading_error_alpha' : ['2.5'],
                  'grading_error_beta' : ['2.5'],
                  'divorce_constant' : ['1000'],
                  'max_strikes' : ['3'],
                  'num_agents' : ['60'],
                  'agent_tolerance_alpha' : ['0.1'],
                  'agent_tolerance_beta' : ['0.1'],
                  'agent_effort_alpha' : ['0.1'],
                  'agent_effort_beta' : ['0.1'],
                  'agent_std_effort' : ['0.5'],
                  'min_agents_per_group' : ['3'],
                  'max_agents_per_group' : ['4'],
                 }

def design_runfile(new_fname, parameter_dict=parameter_dict):
    with open(template_file) as template:
        template_lines = template.readlines()

    parameter_dict['*fname'] = [parameter_dict['*fname'][0] + new_fname.strip('.txt')]
    
    with open(runfile_directory + new_fname, 'w+') as new_file:
        for line in template_lines:
            param = line.split()[0]
            if param not in parameter_dict:
                new_file.write(line)
            else:
                fixer_upper = line.strip('\n').split()[:2]
                new_line = ' '.join(fixer_upper + (parameter_dict[param])) + '\n'
                new_file.write(new_line)
                
def delete_old_files(agent_dir, group_dir):
    agent_files = os.listdir(agent_dir)
    group_files = os.listdir(group_dir)
    for file_name in agent_files:
        file_path = os.path.join(agent_dir, file_name)
        try:
            if os.path.isfile(file_path) or os.path.islink(file_path):
                os.unlink(file_path)  # Remove the file or link
        except Exception as e:
            print(f'Failed to delete {file_path}. Reason: {e}')
    for file_name in group_files:
        file_path = os.path.join(group_dir, file_name)
        try:
            if os.path.isfile(file_path) or os.path.islink(file_path):
                os.unlink(file_path)  # Remove the file or link
        except Exception as e:
            print(f'Failed to delete {file_path}. Reason: {e}')

In [ ]:
# 4 Create the run file
design_runfile('new_model_test.txt', parameter_dict)

In [17]:
# OPTIONAL NUKE OLD FILES
# delete_old_files("Agent_data/", "Group_data/")

In [18]:
# 5 Agent and group analysis directories
result_prefix = "new_model_test"
agent_directory = "Agent_data/"
agent_file_prefix = result_prefix + "new_model_test_Agent-"
group_directory = "Group_data/"
group_file_prefix = result_prefix + "new_model_test_Group-"

In [19]:
# 6 agent and group intake functions
def get_agent_results(agent_file_prefix, time_steps):
    agent_dfs = {}
    for filename in os.listdir(agent_directory):
        file_path = os.path.join(agent_directory, filename)
        # Check if it's a regular file (not a directory)
        if os.path.isfile(file_path):
            temp = open(file_path, 'r')
            temp_lines = temp.readlines()
            num_sections = len(temp_lines)//(time_steps+1)
            df_set = []
            for i in range(num_sections):
                df = pd.read_csv(file_path, skiprows = i*(time_steps+1), nrows = time_steps, comment="#")
                df_set.append(df)
            agent_dfs[filename.strip(agent_file_prefix).strip(".txt")] = df_set
    return agent_dfs

def get_group_results(group_file_prefix, time_steps):
    group_dfs = {}
    for filename in os.listdir(group_directory):
        file_path = os.path.join(group_directory, filename)
        # Check if it's a regular file (not a directory)
        if os.path.isfile(file_path):
            temp = open(file_path, 'r')
            temp_lines = temp.readlines()
            num_sections = len(temp_lines)//(time_steps+1)
            df_set = []
            for i in range(num_sections):
                df = pd.read_csv(file_path, skiprows = i*(time_steps+1), nrows = time_steps, comment="#")
                df_set.append(df)
            group_dfs[filename.strip(group_file_prefix).strip(".txt")] = df_set
    return group_dfs

In [20]:
# 7 get agent and group dfs
agent_dfs = get_agent_results(agent_file_prefix, 100)
group_dfs = get_group_results(group_file_prefix, 100)

In [21]:
print(group_dfs.keys())

dict_keys(['1', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '2', '20', '3', '4', '5', '6', '7', '8', '9'])


In [22]:
display(group_dfs['10'][0])

,group_count,group_payoff
0,0,0.0
1,0,1.0
2,0,1.0
3,0,1.0
4,0,1.0
...,...,...
95,0,1.0
96,0,1.0
97,0,1.0
98,0,1.0


In [25]:
display(agent_dfs['10'][0])

,group_id,id,accumulated_payoff,mean_value,tolerance
0,4,10,1.0,0.231764,1.0
1,4,10,2.0,0.231764,1.0
2,4,10,3.0,0.231764,1.0
3,4,10,4.0,0.231764,1.0
4,4,10,5.0,0.231764,1.0
...,...,...,...,...,...
95,4,10,96.0,0.231764,1.0
96,4,10,97.0,0.231764,1.0
97,4,10,98.0,0.231764,1.0
98,4,10,99.0,0.231764,1.0
